# GPT

Largely inspired by [TinyTorch](https://mlsysbook.ai/tinytorch), the [associated course](https://mlsysbook.ai/assets/downloads/Machine-Learning-Systems.pdf), as well as this excellent [video serie from Karpathy](https://youtu.be/kCc8FmEb1nY) for his nano-gpt.

Thanks Claude for the markdown

In [1]:
# https://github.com/harvard-edge/cs249r_book

import math
import torch
import torch.nn as nn

import numpy as np

To have a basic GPT model we need a bunch of components :

## Tokeniser

There are tons of [different strategies](https://mlsysbook.ai/tinytorch/modules/10_tokenization_ABOUT.html), the simplest being a simple character lookup table from a known corpus. 

For our purpose, can just use one off the shelf if needed : 

In [28]:
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_pretrained("GPT2") 
encoded = tokenizer.encode("Hello world")
print(encoded.ids, encoded.tokens) # Ġ means that there is a space before this token

[15496, 995] ['Hello', 'Ġworld']


## Embeddings 

Turns tokens ids into dense vectors carrying semantic information. Just a lookup table with randomly initialized vectors, that will be learnt as training go. Different initializations schemes are available, the more popular are : 

#### Xavier Normal
$$
w \sim \mathcal{N}\left(0, \frac{2}{n_{\text{in}} + n_{\text{out}}}\right)
$$

#### Xavier Uniform

$$
w \sim \mathcal{U}\left(-\sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}}, \sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}}\right)
$$

#### For Embeddings

If you view the embedding layer as `embedd = one_hot @ W` then $n_{\text{in}} = \text{vocabsize}$ otherwise if you just view it as a lookup table then $n_{\text{in}} = n_{\text{out}} = n_{\text{embed}}$ and the maths simplifies to : 


**Normal**: $w \sim \mathcal{N}\left(0, \frac{1}{n_{\text{embed}}}\right)$

**Uniform**: $w \sim \mathcal{U}\left(-\sqrt{\frac{3}{n_{\text{embed}}}}, \sqrt{\frac{3}{n_{\text{embed}}}}\right)$

Note : Can also directly use pretrained embeddings but it fell out of flavor for modern LLMs.

In [3]:
class EmbeddingLayer(nn.Module):
    """
    Learnable embedding layer that maps token indices to dense vectors.

    This is the fundamental building block for converting discrete tokens
    into continuous representations that neural networks can process.

    APPROACH:
    1. Initialize embedding matrix with random weights (vocab_size, embed_dim)
    2. Implement forward pass as matrix lookup using numpy indexing
    3. Handle batch dimensions correctly
    4. Return parameters for optimization

    EXAMPLE:
    >>> embed = Embedding(vocab_size=100, embed_dim=64)
    >>> tokens = Tensor([[1, 2, 3], [4, 5, 6]])  # batch_size=2, seq_len=3
    >>> output = embed.forward(tokens)
    >>> print(output.shape)
    (2, 3, 64)

    HINTS:
    - Use numpy advanced indexing for lookup: weight[indices]
    - Embedding matrix shape: (vocab_size, embed_dim)
    - Initialize with Xavier/Glorot uniform for stable gradients
    - Handle multi-dimensional indices correctly
    """

    def __init__(self, vocab_size: int, embed_dim: int, init: str = "xavier_uniform"):
        super().__init__()
        
        if init == "xavier_uniform":
            # rand ~ U(0, 1) -> (b - a) * U(0, 1) ~ U(0, b - a) -> (b - a) * U(0, 1) + a ~ U(a, b)
            rdm_uniform = torch.rand(vocab_size, embed_dim)
            limit = math.sqrt(3 / embed_dim)
            embeddings = 2 * limit * rdm_uniform - limit
            
        elif init == "xavier_normal":
            # randn ~ N(0, 1) -> sqrt(a) * N(0, 1) + b ~ N(b, a)
            rdm_normal = torch.randn(vocab_size, embed_dim)
            limit = math.sqrt(1 / embed_dim)
            embeddings = limit * rdm_normal
            
        elif init is None or init is False:
            # N(0, 1) if not specified
            embeddings = torch.randn(vocab_size, embed_dim)
            
        else:
            raise ValueError(f"Expected init in ['xavier_uniform', 'xavier_normal', None, False] but received {init}")

        self.embeddings = nn.Parameter(embeddings, requires_grad=True)

    def forward(self, tokens):
        return self.embeddings[tokens.long()]  # indexes must be int or bool..

## Positional Encodings

Embeddings representing the position of the token in the sequence, as the position also carries meaning.

#### Absolute encoding

In [4]:
class PositionalEncodingLayer(nn.Module):
    """
    Learnable positional encoding layer.

    Adds trainable position-specific vectors to token embeddings,
    allowing the model to learn positional patterns specific to the task.

    APPROACH:
    1. Create embedding matrix for positions: (max_seq_len, embed_dim)
    2. Forward pass: lookup position embeddings and add to input
    3. Handle different sequence lengths gracefully
    4. Return parameters for training

    EXAMPLE:
    >>> pos_enc = PositionalEncoding(max_seq_len=512, embed_dim=64)
    >>> embeddings = Tensor(np.random.randn(2, 10, 64))  # (batch, seq, embed)
    >>> output = pos_enc.forward(embeddings)
    >>> print(output.shape)
    (2, 10, 64)  # Same shape, but now position-aware

    HINTS:
    - Position embeddings shape: (max_seq_len, embed_dim)
    - Use slice [:seq_len] to handle variable lengths
    - Add position encodings to input embeddings element-wise
    - Initialize with smaller values than token embeddings (they're additive)
    """

    def __init__(self, max_seq_len: int, embed_dim: int):
        super().__init__()
        limit = math.sqrt(2 / embed_dim)  # smaller constant for inductive bias : embeddings > position
        xavier_uniform = torch.empty(max_seq_len, embed_dim).uniform_(-limit, limit)  # idiomatic way to init
        self.positional_encoding = nn.Parameter(xavier_uniform)
        self.max_seq_len = max_seq_len

    def forward(self, embeddings):
        seq_len = embeddings.shape[1]
        # early gpt / transformers used absolute pos encoding and required a fixed max length
        if seq_len > self.max_seq_len:
            raise ValueError(
                    f"Sequence length {seq_len} exceeds maximum {self.max_seq_len}"
                )
        # Pytorch broadcasting is smart enough to figure out the missing batch dim
        return embeddings + self.positional_encoding[:seq_len]
        

In [412]:
tokens = torch.Tensor([[1, 2, 3], [4, 5, 6]])  # 2 sequences of 3 tokens

embed = EmbeddingLayer(vocab_size=100, embed_dim=64)
embeddings = embed.forward(tokens)
print(f"Embeddings shape : {embeddings.shape} | min : {embeddings.min()} - max {embeddings.max()}")

pos = PositionalEncodingLayer(max_seq_len=1024, embed_dim=embeddings.shape[-1])
pos_embeddings = pos.forward(embeddings)

print(f"Positional Embeddings shape : {pos_embeddings.shape} | min : {pos_embeddings.min()} - max {pos_embeddings.max()}")

Embeddings shape : torch.Size([2, 3, 64]) | min : -0.2159646600484848 - max 0.21605424582958221
Positional Embeddings shape : torch.Size([2, 3, 64]) | min : -0.38585883378982544 - max 0.3753717243671417


Absolute position encoding isn't used in practice because the encoding changes if you add tokens before the sentence and we're limited by max sequence length, making it hard to extrapolate to new seq lengths

#### Relative encoding

Modern systems use ROPE (or similar) encoding schemes (but not on the embeddings directly, here it's fine for the sake of the exercise). It encodes position as phase, token position scales the rotation angle, frequencies separate scales, and attention dot products convert phase differences into relative position awareness.

- The first token is not rotated because position zero defines the reference frame.
- Each embedding pair has a fixed angular frequency, shared across all tokens.
- Tokens at later positions are rotated by an angle proportional to their position.
- Different frequencies encode relative position at different spatial scales, and their combination allows robust relative positioning over long sequences.



In [5]:
class RotaryPositionalEncodingLayer(nn.Module):
    """
    Rotary Positional Encoding (RoPE) layer.

    IMPORTANT CONCEPTUAL SHIFT:
    ---------------------------
    Unlike learned or sinusoidal positional encodings, RoPE:
    - DOES NOT create a positional embedding matrix
    - DOES NOT add anything to embeddings
    - Instead, it *rotates* embedding vectors in a position-dependent way

    RoPE encodes *relative position* by rotating pairs of embedding
    dimensions using position-dependent angles.

    This layer is typically applied to:
    - Query (Q) and Key (K) tensors in attention
    - NOT to token embeddings directly

    However, for this exercise, we apply it to embeddings to understand
    the mechanism in isolation.

    ------------------------------------------------------------

    INPUT / OUTPUT SHAPE:
    ---------------------
    Input:
        embeddings: Tensor of shape (batch_size, seq_len, embed_dim)

    Output:
        Tensor of same shape (batch_size, seq_len, embed_dim)

    ------------------------------------------------------------

    HIGH-LEVEL IDEA:
    ----------------
    1. Split embedding dimension into pairs:
         (x_0, x_1), (x_2, x_3), ...

    2. Each pair represents a 2D vector

    3. For each position `pos`, rotate each 2D vector by an angle:
         θ(pos, i) = pos / (base ** (2i / embed_dim))

    4. Rotation is done via:
         [cosθ  -sinθ]
         [sinθ   cosθ]

    ------------------------------------------------------------

    WHAT YOU NEED TO PRECOMPUTE:
    ----------------------------
    - A vector of inverse frequencies (inv_freq)
        shape: (embed_dim // 2,)

      Typical formula:
        inv_freq[i] = 1 / (base ** (2i / embed_dim))

      where base is usually 10000.

    - Position indices:
        pos = [0, 1, 2, ..., seq_len - 1]

    ------------------------------------------------------------

    WHAT HAPPENS IN FORWARD():
    --------------------------
    1. Extract seq_len from embeddings
    2. Create position indices [0..seq_len-1]
    3. Compute angles = pos[:, None] * inv_freq[None, :]
    4. Compute sin and cos of angles
    5. Split embeddings into even / odd dimensions
    6. Apply rotation:
         x_even * cos - x_odd * sin
         x_even * sin + x_odd * cos
    7. Re-interleave dimensions
    8. Return rotated embeddings

    ------------------------------------------------------------

    KEY PROPERTIES OF RoPE:
    -----------------------
    - No learned parameters (usually)
    - No max sequence length hard limit
    - Encodes relative position naturally
    - Enables extrapolation to longer sequences
    - Position information survives dot products (Q·K)

    EXAMPLE:
    --------
    >>> rope = PositionalEncodingLayer(embed_dim=64)
    >>> x = torch.randn(2, 10, 64)
    >>> y = rope(x)
    >>> y.shape
    torch.Size([2, 10, 64])
    """

    def __init__(self, embed_dim: int, base: float = 10000.0):
        super().__init__()
        
        if embed_dim % 2 != 0:
            raise ValueError(f"embed_dim must be divisible by 2 but received {embed_dim}")
        
        i = torch.arange(0, embed_dim, 2)  # [0, 2, 4, ..., 62]
        self.inv_freq = 1.0 / (base ** (i / embed_dim))
        

    def forward(self, embeddings: torch.Tensor) -> torch.Tensor:
        """
        Apply RoPE to embeddings.
        
        Args:
            embeddings: (batch_size, seq_len, embed_dim)
        
        Returns:
            Rotated embeddings: (batch_size, seq_len, embed_dim)
        """
        batch_size, seq_len, embed_dim = embeddings.shape
        
        # Step 1: Create position indices [0, 1, 2, ..., seq_len-1]
        positions = torch.arange(seq_len)
        
        # Step 2: Compute rotation angles
        # angles[pos, i] = pos * inv_freq[i]
        angles = positions[:, None] * self.inv_freq[None, :]
        
        # Step 3: Compute sin and cos
        cos_angles, sin_angles = torch.cos(angles), torch.sin(angles)  # (seq_len, embed_dim//2)
        
        # Step 4: Split embeddings into even and odd dimensions
        x_even = embeddings[..., 0::2]  # (batch_size, seq_len, embed_dim//2)
        x_odd = embeddings[..., 1::2]   # (batch_size, seq_len, embed_dim//2)
        
        # Step 5: Apply 2D rotation to each pair
        # Rotation matrix: [cos  -sin]  applied to [x_even]
        #                  [sin   cos]             [x_odd ]
        # Result:
        #   x_even_rotated = x_even * cos - x_odd * sin
        #   x_odd_rotated  = x_even * sin + x_odd * cos
        x_even_rotated = x_even * cos_angles - x_odd * sin_angles
        x_odd_rotated = x_even * sin_angles + x_odd * cos_angles
        
        # Step 6: Interleave back to original dimension order
        # Stack along new dimension then flatten
        x_out = torch.stack([x_even_rotated, x_odd_rotated], dim=-1)
        x_out = x_out.flatten(-2)  # (batch_size, seq_len, embed_dim)
        
        return x_out

In [6]:
class Embedding(nn.Module):
    """
    Complete embedding system combining token and positional embeddings.

    This is the component that handles the full embedding
    pipeline used in transformers and other sequence models.

    APPROACH:
    1. Combine token embedding + positional encoding
    2. Support both learned and sinusoidal position encodings
    3. Handle variable sequence lengths gracefully
    4. Add optional embedding scaling (Transformer convention)

    EXAMPLE:
    >>> embed_layer = EmbeddingLayer(
    ...     vocab_size=50000,
    ...     embed_dim=512,
    ...     max_seq_len=2048,
    ...     pos_encoding='learned'
    ... )
    >>> tokens = Tensor([[1, 2, 3], [4, 5, 6]])
    >>> output = embed_layer.forward(tokens)
    >>> print(output.shape)
    (2, 3, 512)

    HINTS:
    - First apply token embedding, then add positional encoding
    - Handle both 2D (batch, seq) and 1D (seq) inputs gracefully
    - Scale embeddings by sqrt(embed_dim) if requested (transformer convention)
    """

    def __init__(
        self,
        vocab_size: int,
        embed_dim: int,
        max_seq_len: int = 512,
        scale_embeddings: bool = False,
        positional_embedding = "absolute"  # rotary
    ):
        super().__init__()
        self.embed_dim = embed_dim
        self.scale_embeddings = scale_embeddings
        self.embedding = EmbeddingLayer(vocab_size, self.embed_dim)
        if positional_embedding == "absolute":
            self.positional_embedding = PositionalEncodingLayer(max_seq_len, self.embed_dim)
        elif positional_embedding == "rotary":
            self.positional_embedding = RotaryPositionalEncodingLayer(self.embed_dim, 10000)
        elif positional_embedding is None or positional_embedding is False:
            self.positional_embedding = nn.Identity()
        else:
            raise ValueError(f"positional_embedding only supports ['absolute', 'rotary', 'False', 'None'] but received {positional_embedding}")

    def forward(self, tokens):
        if len(tokens.shape) == 1:
            tokens = tokens.unsqueeze(0)  # add batch dim

        embeddings = self.embedding(tokens)
        if self.scale_embeddings:
            # transformer convention, helps with gradient stability
            # and helps embedding dominate vs positions
            embeddings *= math.sqrt(self.embed_dim) 

        return self.positional_embedding(embeddings)

## Attention Mechanism

Now that we have a way to encode our words we can compute stuff

### Core Intuition
Attention allows each token to **selectively focus** on other tokens in the sequence. Instead of treating all positions equally, the model learns which positions are most relevant for processing each token.

### Mathematical Formulation

Given an input sequence of embeddings $\mathbf{X} \in \mathbb{R}^{n \times d}$ where $n$ is sequence length and $d$ is embedding dimension:

#### 1. **Linear Projections**
Transform input into three representations:
$$\mathbf{Q} = \mathbf{X}\mathbf{W}^Q, \quad \mathbf{K} = \mathbf{X}\mathbf{W}^K, \quad \mathbf{V} = \mathbf{X}\mathbf{W}^V$$

where $\mathbf{W}^Q, \mathbf{W}^K, \mathbf{W}^V \in \mathbb{R}^{d \times d_k}$ are learned weight matrices.

- **Query** ($\mathbf{Q}$): "What am I looking for?"
- **Key** ($\mathbf{K}$): "What do I contain?"
- **Value** ($\mathbf{V}$): "What information do I carry?"

*Note : it's called self-attention because all 3 are computed from same input $X$*

#### 2. **Attention Scores**
Compute similarity between queries and keys:
$$\mathbf{S} = \frac{\mathbf{Q}\mathbf{K}^T}{\sqrt{d_k}} \in \mathbb{R}^{n \times n}$$

- Dot product measures compatibility between query $i$ and key $j$
- Scaling by $\sqrt{d_k}$ prevents extremely large values (stabilizes gradients)
- Entry $S_{ij}$ = "how much should token $i$ attend to token $j$?"

#### 3. **Attention Weights**
Normalize scores into probabilities:
$$\mathbf{A} = \text{softmax}(\mathbf{S}) = \text{softmax}\left(\frac{\mathbf{Q}\mathbf{K}^T}{\sqrt{d_k}}\right)$$

where softmax is applied row-wise: $A_{ij} = \frac{\exp(S_{ij})}{\sum_{k=1}^{n} \exp(S_{ik})}$

- Each row sums to 1: $\sum_{j=1}^{n} A_{ij} = 1$
- $A_{ij}$ = probability that token $i$ attends to token $j$

#### 4. **Weighted Sum**
Aggregate values using attention weights:
$$\mathbf{Z} = \mathbf{A}\mathbf{V} \in \mathbb{R}^{n \times d_k}$$

Each output $\mathbf{z}_i$ is a weighted combination: $\mathbf{z}_i = \sum_{j=1}^{n} A_{ij} \mathbf{v}_j$

### Complete Formula (Scaled Dot-Product Attention)

$$\boxed{\text{Attention}(\mathbf{Q}, \mathbf{K}, \mathbf{V}) = \text{softmax}\left(\frac{\mathbf{Q}\mathbf{K}^T}{\sqrt{d_k}}\right)\mathbf{V}}$$

### Multi-Head Attention

Split features into bags and run $h$ attention operations in parallel with different learned projections (1 attention per bag of features):

$$\text{head}_i = \text{Attention}(\mathbf{X}\mathbf{W}^Q_i, \mathbf{X}\mathbf{W}^K_i, \mathbf{X}\mathbf{W}^V_i)$$

$$\text{MultiHead}(\mathbf{X}) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h)\mathbf{W}^O$$

where $\mathbf{W}^O \in \mathbb{R}^{hd_k \times d}$ projects concatenated heads back to model dimension.

**Benefit**: Each head can learn different attention patterns (e.g., syntax, semantics, positional relationships).

### Key Properties

- **Permutation Equivariant**: Without positional encodings, shuffling input tokens shuffles output identically
- **Parallelizable**: All positions computed simultaneously (unlike RNNs)
- **Long-Range Dependencies**: Any token can directly attend to any other token in $O(1)$ operations
- **Complexity**: $O(n^2 d)$ time and $O(n^2)$ memory for sequence length $n$


### Differentiable Hash-table (python dict)

Can also view this attention mechanism as a "soft" python dictionary where the key doesn't need to match exactly and the dictionary returns a weighted blend of multiple values


### Causal mask
- Prevents each token from attending to future tokens by masking out positions to the right in the attention matrix.
- It enforces autoregressive behavior: the model can only use past and present information when predicting the next token (typical for text generation).
- Without it the model would “cheat” during training by seeing the future and would fail at generation time.

In [7]:
class MultiHeadAttention(nn.Module):
    """
    Multi-head attention mechanism.

    Runs multiple attention heads in parallel, each learning different relationships.
    
    APPROACH:
        1. Validate that embed_dim is divisible by num_heads
        2. Calculate head_dim (embed_dim // num_heads)
        3. Create linear layers for Q, K, V projections
        4. Create output projection layer
        5. Store configuration parameters

        Args:
            embed_dim: Embedding dimension (d_model)
            num_heads: Number of parallel attention heads

        EXAMPLE:
        >>> mha = MultiHeadAttention(embed_dim=512, num_heads=8)
        >>> mha.head_dim  # 64 (512 / 8)
        >>> len(mha.parameters())  # 4 linear layers * 2 params each = 8 tensors

        HINTS:
        - head_dim = embed_dim // num_heads must be integer
        - Need 4 Linear layers: q_proj, k_proj, v_proj, out_proj
        - Each projection maps embed_dim → embed_dim
    
    """

    def __init__(self, embed_dim: int, num_heads: int, causal: bool = True):
        super().__init__()

        # 1. Validate configuration
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embed_dim ({embed_dim}) must be divisible by num_heads ({num_heads})."
            )
        
        # 2. Store parameters
        self.causal = causal  # mask futur tokens ?
        self.embed_dim = embed_dim          # d_model
        self.num_heads = num_heads          # H
        self.head_dim = embed_dim // num_heads  # d_head

        # 3. Linear projections for Q, K, V
        # Each maps: (B, N, embed_dim) → (B, N, embed_dim)
        self.q_proj = nn.Linear(embed_dim, embed_dim, bias=False)
        self.k_proj = nn.Linear(embed_dim, embed_dim, bias=False)
        self.v_proj = nn.Linear(embed_dim, embed_dim, bias=False)

        # 4. Output projection (after concatenating heads)
        self.out_proj = nn.Linear(embed_dim, embed_dim)


    def forward(self, x):
        B, N, _ = x.shape
        # split features into num_heads bags of head_dim features
        Q = self.q_proj(x).reshape(B, N, self.num_heads, self.head_dim)
        V = self.v_proj(x).reshape(B, N, self.num_heads, self.head_dim)
        K = self.k_proj(x).reshape(B, N, self.num_heads, self.head_dim)

        # transpose to (batch, num_heads, seq, head_dim) for parallel processing
        Q = Q.transpose(1, 2)
        K = K.transpose(1, 2)
        V = V.transpose(1, 2)

        scores = Q @ K.transpose(-2, -1) / math.sqrt(self.head_dim)  # (batch, num_heads, seq, seq)
        
        if self.causal:
            auto_regressive_mask = torch.triu(
                torch.ones(N, N, device=x.device),
                diagonal=1
            ).bool()
            # fill upper triangle with -inf value -> 0 after softmax
            scores = scores.masked_fill(auto_regressive_mask, float('-inf'))

        # softmax along keys
        scores = torch.softmax(scores, -1)
        output = scores @ V
        
        # concatenate the heads and final proj
        return self.out_proj(output.transpose(1, 2).contiguous().view(B, N, self.embed_dim))

## Layer Normalization vs Batch Normalization

### Why Layer Norm for Transformers?

#### Normalization Axis Comparison

**Batch Normalization**: Normalizes across the **batch dimension**
$$\mu_j = \frac{1}{B}\sum_{i=1}^{B} x_{ij}, \quad \sigma_j^2 = \frac{1}{B}\sum_{i=1}^{B} (x_{ij} - \mu_j)^2$$

For input shape `(B, N, D)` → computes statistics over `B` (batch) for each feature `j`

**Layer Normalization**: Normalizes across the **feature dimension**
$$\mu_i = \frac{1}{D}\sum_{j=1}^{D} x_{ij}, \quad \sigma_i^2 = \frac{1}{D}\sum_{j=1}^{D} (x_{ij} - \mu_i)^2$$

For input shape `(B, N, D)` → computes statistics over `D` (features) for each sample `i`

#### Visual Comparison
```
Input shape: (Batch=2, SeqLen=3, Features=4)

Batch Norm:           Layer Norm:
[[ 1  2  3  4]        [[ 1  2  3  4]
 [ 5  6  7  8]         [ 5  6  7  8]
 [ 9 10 11 12]         [ 9 10 11 12]
 [13 14 15 16]         [13 14 15 16]
 [17 18 19 20]         [17 18 19 20]
 [21 22 23 24]]        [21 22 23 24]]
 
 ↓ Normalize           → Normalize
 per column            per row
```

### Why Layer Norm Wins for Transformers

| Issue | Batch Norm Problem | Layer Norm Solution |
|-------|-------------------|---------------------|
| **Variable Sequence Lengths** | Statistics change with padding | Each sequence normalized independently |
| **Small Batch Sizes** | Unstable statistics (mean/var unreliable) | Statistics computed per sample |
| **Train/Test Discrepancy** | Needs running statistics, different behavior | Same computation always |
| **Sequential Processing** | Can't normalize one sample at inference | Works naturally for online inference |

#### Key Insight
> Transformers process **variable-length sequences** with **self-attention**. Each token's representation should be normalized based on **its own feature distribution**, not compared to other samples in the batch.

### Biased vs Unbiased Variance

#### The Math

**Biased (Population) Variance**: $\sigma^2 = \frac{1}{N}\sum_{i=1}^{N}(x_i - \mu)^2$

**Unbiased (Sample) Variance**: $s^2 = \frac{1}{N-1}\sum_{i=1}^{N}(x_i - \mu)^2$

#### PyTorch Uses Biased Variance (`unbiased=False`)

**Ratio**: $\frac{\sigma_{\text{unbiased}}^2}{\sigma_{\text{biased}}^2} = \frac{D}{D-1} = \frac{512}{511} \approx 1.002$

**Practical Impact**:
- For **large feature dimensions** (D=512, 768, 1024): difference is **<0.2%** → negligible
- Biased estimator is the **population parameter** we actually want to normalize by
- Using `unbiased=True` would slightly **over-normalize** (divide by slightly larger σ)

#### Why Use Biased?

1. **Consistency**: We're normalizing the **current data**, not estimating a population parameter
2. **Stability**: Dividing by N (not N-1) is more stable when N is small
3. **Standard Practice**: PyTorch's `nn.LayerNorm` uses biased variance
4. **Negligible Difference**: For D ≫ 1, the correction factor D/(D-1) ≈ 1


### Complete LayerNorm Formula

$$\text{LayerNorm}(x) = \gamma \odot \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta$$

where:
- $\mu = \frac{1}{D}\sum_{j=1}^{D} x_j$ (mean over features)
- $\sigma^2 = \frac{1}{D}\sum_{j=1}^{D} (x_j - \mu)^2$ (biased variance over features)
- $\gamma, \beta$ are learnable parameters (initialized to 1 and 0)
- $\epsilon$ small value to prevent division by zero

In [8]:
class LayerNorm(nn.Module):
    """
    Layer Normalization for transformer blocks.

    Normalizes across the feature dimension (last axis) for each sample independently,
    unlike batch normalization which normalizes across the batch dimension.

    Mathematical Formula:
    output = (x - μ) / σ * γ + β
    
    where:
      μ = mean(x, axis=features)     # Mean across feature dimension
      σ = sqrt(var(x) + ε)          # Standard deviation + small epsilon
      γ = learnable scale parameter  # Initialized to 1.0
      β = learnable shift parameter  # Initialized to 0.0
      
    Why Layer Norm Works:
    
    Independence: Each sample normalized independently (good for variable batch sizes)
    Stability: Prevents internal covariate shift that breaks training
    Gradient Flow: Helps gradients flow better through deep networks
    """

    def __init__(self, normalized_shape, eps=1e-5):
        """
        Initialize LayerNorm with learnable parameters.

        APPROACH:
        1. Store the shape to normalize over (usually embed_dim)
        2. Initialize learnable scale (gamma) and shift (beta) parameters
        3. Set small epsilon for numerical stability
        """
        super().__init__()

        self.normalized_shape = normalized_shape
        self.eps = eps

        # Learnable parameters
        self.gamma = nn.Parameter(torch.ones(normalized_shape))
        self.beta = nn.Parameter(torch.zeros(normalized_shape))

    def forward(self, x):
        """
        Apply layer normalization.

        APPROACH:
        1. Compute mean and variance across the last dimension
        2. Normalize: (x - mean) / sqrt(variance + eps)
        3. Apply learnable scale and shift: gamma * normalized + beta
        """

        mean = x.mean(-1, keepdim=True)
        # var() divides by N-1 but torch implementation uses population var i.e divides by N
        std = torch.sqrt(x.var(-1, keepdim=True, unbiased=False) + self.eps) # <=> np.sqrt(((x - mean) ** 2).mean(-1, keepdim=True))
        x = (x - mean) / std
        return x * self.gamma + self.beta

## MLP block

### Attention
Mixes information across tokens. Each token dynamically selects which other tokens to read from, based on content. This enables context, dependency modeling, and information routing over the sequence.

### MLP (Feed-Forward Network)
Mixes information within a token. It non-linearly recombines and transforms features of each token independently, increasing representational power.

### Why both
Attention provides communication between tokens; the MLP provides computation on token features.
Without attention, tokens can’t interact. Without MLPs, the model is mostly linear and weak.

#### Why GELU instead of ReLU?

```
# ReLU: Hard cutoff at 0
relu(x) = max(0, x)

# GELU: Smooth, probabilistic gating
gelu(x) = x * Φ(x)  # Φ is standard normal CDF
```
*Benefits of GELU:*

- Smoother gradients (no sharp corner at x=0)
- Better performance in language models (empirically)
- Can be thought of as "stochastic regularization"

#### Why 4x expansion ? 

2x too small, 8x too slow, 4x works well


#### Why No Activation After Second Layer?

```
# Transformer block structure:
output = x + MLP(LayerNorm(x))  # Residual connection
        ↑
        └─ Need to preserve residual path
```

Adding GELU after linear2, would force all outputs to be non-negative, which:

- Limits expressiveness of the residual stream
- Can cause gradient flow issues
- Breaks the "smooth information highway" of residual connections

In [9]:
class MLP(nn.Module):
    """
    Multi-Layer Perceptron (Feed-Forward Network) for transformer blocks.

    Standard pattern: Linear -> GELU -> Linear with expansion ratio of 4:1.
    """

    def __init__(self, embed_dim, hidden_dim=None, dropout_prob=0.1):
        """
        Initialize MLP with two linear layers.

        APPROACH:
        1. First layer expands from embed_dim to hidden_dim (usually 4x larger)
        2. Second layer projects back to embed_dim
        3. Use GELU activation (smoother than ReLU, preferred in transformers)

        EXAMPLE:
        >>> mlp = MLP(512)  # Will create 512 -> 2048 -> 512 network
        >>> x = Tensor(np.random.randn(2, 10, 512))
        >>> output = mlp.forward(x)
        >>> assert output.shape == (2, 10, 512)

        """
        super().__init__()

        if hidden_dim is None:
            hidden_dim = 4 * embed_dim  # Standard transformer expansion

        # params
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim

        # actual layers, bias on by default, some architectures turn it off here
        self.linear1 = nn.Linear(embed_dim, hidden_dim)
        self.act = nn.GELU()
        self.linear2 = nn.Linear(hidden_dim, embed_dim)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = self.act(self.linear1(x))
        x = self.linear2(x)  # No GELU here to not mess with residual paths x + MLP(x)
        return self.dropout(x)

### Transformer Block

Now that we have each component, we can code the actual computation block.

Tl;dr :

#### Transformer block
Applies self-attention followed by an MLP, each wrapped with residual connections (and normalization).

- Self-attention: lets tokens exchange information across the sequence.
- MLP: non-linearly transforms each token’s features.
- Residual connections: preserve information and stabilize training.

Attention enables communication, the MLP enables computation, and residuals allow stacking many layers without losing signal.

#### Why pre-norm
Empirically found in a paper to be better for stability than applying LayerNorm after.

#### Why stack many blocks
Each block performs a small amount of communication (attention) and computation (MLP). Stacking many blocks lets the model build increasingly abstract, long-range, and hierarchical representations. One block is shallow; depth gives expressive power.

In [10]:
class TransformerBlock(nn.Module):
    """
    Complete Transformer Block with self-attention, MLP, and residual connections.

    This is the core building block of GPT and other transformer models.
    Each block processes the input sequence and passes it to the next block.
    """

    def __init__(self, embed_dim, num_heads, mlp_ratio=4, dropout_prob=0.1, causal=True):
        """
        Initialize a complete transformer block.
        
        APPROACH:
        1. Multi-head self-attention for sequence modeling
        2. First layer normalization (pre-norm architecture)
        3. MLP with specified expansion ratio
        4. Second layer normalization

        TRANSFORMER BLOCK ARCHITECTURE:
        x → LayerNorm → MultiHeadAttention → + (residual) →
            LayerNorm → MLP → + (residual) → output

        EXAMPLE:
        >>> block = TransformerBlock(embed_dim=512, num_heads=8)
        >>> x = Tensor(np.random.randn(2, 10, 512))  # (batch, seq, embed)
        >>> output = block.forward(x)
        >>> assert output.shape == (2, 10, 512)

        HINT: We use pre-norm architecture (LayerNorm before attention/MLP)
        """

        super().__init__()

        self.layernorm1 = LayerNorm(embed_dim)
        self.mha = MultiHeadAttention(embed_dim, num_heads, causal)  # causal = masking out tokens
        self.layernorm2 = LayerNorm(embed_dim)
        self.mlp = MLP(embed_dim, mlp_ratio * embed_dim, dropout_prob)

    def forward(self, x):
        """
        Forward pass through transformer block.

        APPROACH:
        1. Apply layer norm, then self-attention, then add residual
        2. Apply layer norm, then MLP, then add residual
        3. Return the transformed sequence

        COMPUTATION FLOW:
        x → ln1 → attention → + x → ln2 → mlp → + → output

        RESIDUAL CONNECTIONS:
        These are crucial for training deep networks - they allow gradients
        to flow directly through the network during backpropagation.

        HINT: 
        - Store intermediate results to add residual connections properly
        - Don't forget masking
        """
        x1 = self.layernorm1(x)
        x2 = self.mha(x1) + x  # residual path

        x3 = self.layernorm2(x2)
        x3 = self.mlp(x3) + x2  # residual path
        return x3

# Putting it all together : GPT

Words -> tokens -> embeddings + position -> stack of transformer blocks -> prediction layer

One thing worth mentioning is the weight tying between the embedding table and the final linear layer. It is explicitely asked in the docstring but not actually implemented in the [official solution](https://hub.2i2c.mybinder.org/user/harvard-edge-cs249r_book-2qly0zim/doc/tree/tinytorch/modules/13_transformers/13_transformers.ipynb), however it is present in [nano-gpt](https://github.com/karpathy/nanoGPT/blob/3adf61e154c3fe3fca428ad6bc3818b27a3b8291/model.py#L138)

I think it makes sense, you want the embedding table to perform token id -> embedding but also the reverse operation (if you transpose it) ie vector living in same embedding space -> ~~~token id

In [11]:
class GPT(nn.Module):
    """
    Complete GPT (Generative Pre-trained Transformer) model.

    This combines embeddings, positional encoding, multiple transformer blocks,
    and a language modeling head for text generation.
    """

    def __init__(self, vocab_size, embed_dim, num_layers, num_heads, max_seq_len=1024):
        """
        Initialize complete GPT model.

        APPROACH:
        1. Token embedding layer to convert tokens to vectors
        2. Positional embedding to add position information
        3. Stack of transformer blocks (the main computation)
        4. Final layer norm and language modeling head

        GPT ARCHITECTURE:
        tokens → embedding → + pos_embedding →
                transformer_blocks → layer_norm → lm_head → logits

        EXAMPLE:
        >>> model = GPT(vocab_size=1000, embed_dim=256, num_layers=6, num_heads=8)
        >>> tokens = Tensor(np.random.randint(0, 1000, (2, 10)))  # (batch, seq)
        >>> logits = model.forward(tokens)
        >>> assert logits.shape == (2, 10, 1000)  # (batch, seq, vocab)

        HINTS:
        - Positional embeddings are learned, not fixed sinusoidal
        - Final layer norm stabilizes training
        - Language modeling head shares weights with token embedding (tie_weights)
        """
        super().__init__()

        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.max_seq_len = max_seq_len
        
        self.embedding = Embedding(
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            max_seq_len=self.max_seq_len,
            scale_embeddings=True,
            positional_embedding="absolute"       
        )
        self.blocks = nn.ModuleList([TransformerBlock(embed_dim, num_heads) for _ in range(num_layers)])
        self.ln = LayerNorm(embed_dim)
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
        # mental naming lol
        # but basically you want the embedding table to perform both operations words -> embed and embed -> prediction
        # so we jointly learn last and first layer
        # bias = false, to match the lookup table structure
        self.embedding.embedding.embeddings = self.lm_head.weight
       
    def forward(self, tokens):
        x = self.embedding(tokens)  # tokens -> emb + pos enc
        for b in self.blocks:
            x = b(x)  # iteratively refines features from initial embeddings
        features = self.ln(x)  # normalized to stabilize training
        return self.lm_head(features)

    @torch.no_grad()
    def generate(self, prompt_tokens, max_new_tokens=50, temperature=1.0):
        """
        Generate text autoregressively.

        APPROACH:
        1. Start with prompt tokens
        2. For each new position:
           - Run forward pass to get logits
           - Sample next token from logits
           - Append to sequence
        3. Return generated sequence

        AUTOREGRESSIVE GENERATION:
        At each step, the model predicts the next token based on all
        previous tokens. This is how GPT generates coherent text.

        EXAMPLE:
        >>> model = GPT(vocab_size=100, embed_dim=64, num_layers=2, num_heads=4)
        >>> prompt = Tensor([[1, 2, 3]])  # Some token sequence
        >>> generated = model.generate(prompt, max_new_tokens=5)
        >>> assert generated.shape[1] == 3 + 5  # original + new tokens

        HINT: Use np.random.choice with temperature for sampling
        """
        self.eval()
        current_tokens = prompt_tokens.clone()
        
        with torch.no_grad():
            for _ in range(max_new_tokens):
                # Forward pass
                logits = self(current_tokens)          # (B, T, V)
        
                # Last token logits
                last_logits = logits[:, -1, :]         # (B, V)
        
                # Temperature
                scaled_logits = last_logits / temperature
        
                # Softmax
                probs = torch.softmax(scaled_logits, dim=-1)
        
                # Sample next token (Torch-native)
                next_token = torch.multinomial(probs, num_samples=1)  # (B, 1)
        
                # Append
                current_tokens = torch.cat([current_tokens, next_token], dim=1)
        
        return current_tokens

# Sanity check

In [452]:
gpt = GPT(vocab_size=1000, embed_dim=256, num_layers=6, num_heads=8)
logits = gpt(tokens)
print(tokens.shape, logits.shape, gpt.lm_head.weight is gpt.embedding.embedding.embeddings)

gpt.eval()
tokens = torch.tensor([[1, 2, 3, 4]])
logits = gpt(tokens)

tokens2 = torch.tensor([[1, 2, 999, 4]])
logits2 = gpt(tokens2)

# need to be identical
print(torch.abs(logits[:, :2] - logits2[:, :2]).mean())

out = model.generate(tokens, 5)
print(out)

torch.Size([1, 4]) torch.Size([1, 4, 1000]) True
tensor(0., grad_fn=<MeanBackward0>)
tensor([[ 1,  2,  3,  4, 10, 10, 10, 10, 10]])


In [27]:
from torch.utils.data import Dataset, DataLoader

class CharDataset(Dataset):
    def __init__(self, text, block_size):
        self.block_size = block_size
        chars = sorted(list(set(text)))
        self.stoi = {ch: i for i, ch in enumerate(chars)}
        self.itos = {i: ch for ch, i in self.stoi.items()}
        self.vocab_size = len(chars)

        self.data = torch.tensor([self.stoi[c] for c in text], dtype=torch.long)

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        x = self.data[idx : idx + self.block_size]
        y = self.data[idx + 1 : idx + self.block_size + 1]
        return x, y


def train(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0.0

    for x, y in dataloader:
        x = x.to(device)
        y = y.to(device)

        logits = model(x)                  # (B, T, V)
        loss = torch.nn.functional.cross_entropy(
            logits.view(-1, logits.size(-1)),
            y.view(-1)
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)


device = "cuda" if torch.cuda.is_available() else "cpu"

text = "hello world\n" * 100
block_size = 8
batch_size = 32

dataset = CharDataset(text, block_size)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = GPT(
    vocab_size=dataset.vocab_size,
    embed_dim=128,
    num_layers=8,
    num_heads=4,
).to(device)

optimizer = torch.optim.AdamW(model.parameters())

for epoch in range(50):
    loss = train(model, loader, optimizer, device)
    print(f"epoch {epoch:02d} | loss {loss:.4f}")

epoch 00 | loss 0.4297
epoch 01 | loss 0.1400
epoch 02 | loss 0.1117
epoch 03 | loss 0.0660
epoch 04 | loss 0.0630
epoch 05 | loss 0.0636
epoch 06 | loss 0.0562
epoch 07 | loss 0.0564
epoch 08 | loss 0.0556
epoch 09 | loss 0.0524
epoch 10 | loss 0.0526
epoch 11 | loss 0.0526
epoch 12 | loss 0.0529
epoch 13 | loss 0.0509
epoch 14 | loss 0.1056
epoch 15 | loss 0.0718
epoch 16 | loss 0.0555
epoch 17 | loss 0.0712
epoch 18 | loss 0.0698
epoch 19 | loss 0.0614
epoch 20 | loss 0.0557
epoch 21 | loss 0.0516
epoch 22 | loss 0.0524
epoch 23 | loss 0.0519
epoch 24 | loss 0.0517
epoch 25 | loss 0.0510
epoch 26 | loss 0.0501
epoch 27 | loss 0.0524
epoch 28 | loss 0.0514
epoch 29 | loss 0.0502
epoch 30 | loss 0.0523
epoch 31 | loss 0.0501
epoch 32 | loss 0.0507
epoch 33 | loss 0.0497
epoch 34 | loss 0.0503
epoch 35 | loss 0.0506
epoch 36 | loss 0.0516
epoch 37 | loss 0.0503
epoch 38 | loss 0.0491
epoch 39 | loss 0.0505
epoch 40 | loss 0.0499
epoch 41 | loss 0.0506
epoch 42 | loss 0.0494
epoch 43 | 

In [24]:
start_token = dataset.stoi["h"]

prompt = torch.tensor(
    [[start_token]],
    dtype=torch.long,
    device=device
)
out = model.generate(prompt, max_new_tokens=20, temperature=0.7)
tokens = out[0].tolist()
text = "".join(dataset.itos[i] for i in tokens)
print(text)

hello world
helllorlo


In [26]:
# We're good !
for c in "hel":
    prompt = torch.tensor([[dataset.stoi[c]]], device=device)
    print(c, "→", "".join(dataset.itos[i] for i in model.generate(prompt, 10, temperature=0.7)[0].tolist()))

h → hello world
e → ello world

l → lo world
he


## TO DO : 

- ROPE for K, V, Q
- Top k sampling
- Training on a real problem to see how far we can push current model
- Revisit markdown / maths
- Check newer architectures / design choices (https://github.com/lucidrains git is a gold mine)